# Домашняя работа

Будем работать с датасетом подержанных машин https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes.

Задача - построить прогноз цены машины по ее данным (год выпуска, пробег, расход, размер двигателя, тип коробки передач и тд). 

## Easy

Построить прогноз по одной марке машин при помощи решающих деревьев, взять только числовые признаки. В качестве метрики взять `r2_score`. Оценить важность признаков, проинтерпретировать модель.

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

## Hard

Собрать датасет по одной модели на любом агрегаторе объявлений (модель должна быть из датасета). Сравнить какие признаки наиболее важны на нашем рынке, по сравнению с рынком в UK.

Сервисы не любят парсеры, поэтому скорее всего в автоматическом режиме не получится распарсить много данных. По-крайней мере, у меня не получилось как-то быстро обойти капчу. Поэтому:

- Можно вручную вбить в эксельку данные по нескольким десяткам машин
- Можно скачать несколько html и написать скрипт для их парсинга (можно использовать beautiful soup) - в таком подходе ожидаю больше сотни примеров
- Можно найти агрегатор без капчи или апишку (примеры - телеграм каналы, VK API) и собрать данные оттуда

In [1]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, make_scorer

## Easy

In [2]:
data = pd.read_csv("../data/cars_data/bmw.csv")
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["price", "model", "transmission", "fuelType"]), data["price"])

In [4]:
model = RandomForestRegressor(n_estimators=200)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200)

In [5]:
preds = model.predict(X_test)
print(r2_score(preds, y_test))
print(pd.DataFrame(zip(X_train.columns, model.feature_importances_)))

0.8758323771623042
            0         1
0        year  0.456512
1     mileage  0.108522
2         tax  0.015730
3         mpg  0.150928
4  engineSize  0.268309


# Выводы:
Как можно заметить, наиболее важным признаком при выборе машины является год производства. Также довольно важным признаком является потребление топлива (mpg) и объем двигателя. Что интересно, пробег не так важен. 
Итого, R^2 score у текущей модели колеблется около 0.85, что в целом довольно хорошо (учитывая, что данная метрика может быть и отрицательной).

Из интересного -- я эксперимента ради обучил ещё DecisionTreeClassifier, получил метрику около 0.8, при этом значимыми признаками были уже пробег и потребление топлива, у остальных признаков значимость была около 0.05.

## Medium

In [6]:
def read_csv_files(directory):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            dfs.append(df)
    return dfs

def merge_dataframes(dfs):
    
    common_columns = set(dfs[0].columns)
    for df in dfs[1:]:
        common_columns = common_columns.intersection(df.columns)
    
    common_columns = list(common_columns)
    filtered_dfs = [df.loc[:, common_columns] for df in dfs]
    concat_dfs = pd.concat(filtered_dfs)
    
    return concat_dfs

dataframes = read_csv_files('../data/cars_data')

merged_dataframe = merge_dataframes(dataframes)
dummy_df = pd.get_dummies(data=merged_dataframe, columns=["transmission", "fuelType", "model"], drop_first=True)

In [7]:
# Данных слишком много, одна модель обучается около 13 минут, поэтому оставим только часть данных
dummy_df = dummy_df.sample(int(len(dummy_df)*0.1), random_state=42)
len(dummy_df)

10854

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dummy_df.drop(columns=["price"]), dummy_df["price"], test_size=0.2)
X_train.head()

,mileage,engineSize,year,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol,...,model_ Z3,model_ Z4,model_ Zafira,model_ Zafira Tourer,model_ i3,model_ i8,model_180,model_200,model_220,model_230
8855,14168,2.2,2019,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1978,5594,2.0,2019,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1081,34425,2.0,2017,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11549,13800,2.7,2002,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5251,48094,2.0,2016,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
rf_model = RandomForestRegressor()

In [10]:
rf_params = {
    'n_estimators': [5, 100, 500],
    'max_depth': [5, 100, 150],
    'max_leaf_nodes': [None, 10, 100]
}

In [11]:
scorer = make_scorer(r2_score)

In [12]:
search = RandomizedSearchCV(rf_model, rf_params, n_iter=10, scoring=scorer)
search.fit(X_train, y_train)
search.best_estimator_

RandomForestRegressor(max_depth=150, n_estimators=500)

In [13]:
search.cv_results_

{'mean_fit_time': array([ 0.05713673,  2.19047103, 12.76825218,  0.96782122,  2.59898152,
         0.05108895, 11.47801847,  6.00432639,  0.06290264,  0.94886365]),
 'std_fit_time': array([0.00640557, 0.07823531, 0.11223944, 0.00663375, 0.07692694,
        0.00429959, 0.1703727 , 0.04461225, 0.00024498, 0.01604115]),
 'mean_score_time': array([0.        , 0.01219988, 0.17887826, 0.00685105, 0.0333147 ,
        0.00310788, 0.05035267, 0.03788815, 0.        , 0.00932913]),
 'std_score_time': array([0.        , 0.00616193, 0.01346161, 0.00844447, 0.00797984,
        0.00621576, 0.00629572, 0.00755012, 0.        , 0.00766593]),
 'param_n_estimators': masked_array(data=[5, 100, 500, 100, 100, 5, 500, 500, 5, 100],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_leaf_nodes': masked_array(data=[None, 100, None, 10, None, 10, 100, 100, 100, 10],
              mask=[Fal

# Cравним модели!
Для этого я попарно проделаю тест Стьюдента с p-value=0.05 для каждого результата в фолдах. 


In [14]:
from scipy.stats import ttest_ind

splits_test_scores = [search.cv_results_[f"split{i}_test_score"] for i in range(5)]
cvs = tuple(zip(*splits_test_scores))

for n in range(10):
    models_beated = len([ttest_ind(cvs[n], cvs[i], alternative='greater')[1] for i in range(10) if ttest_ind(cvs[n], cvs[i], alternative='greater')[1] <= 0.05])
    print(f"Model {n} got statistically better results than {models_beated} models.")

Model 0 got statistically better results than 3 models.
Model 1 got statistically better results than 6 models.
Model 2 got statistically better results than 8 models.
Model 3 got statistically better results than 0 models.
Model 4 got statistically better results than 8 models.
Model 5 got statistically better results than 0 models.
Model 6 got statistically better results than 6 models.
Model 7 got statistically better results than 3 models.
Model 8 got statistically better results than 3 models.
Model 9 got statistically better results than 0 models.


Итого, после такого сравнения самой лучшей оказалась модель 2 (разделяя почетное место с моделью 4).
Параметры у них похожи -- нет ограничений по числу листьев, только немного разнится глубина и количество деревьев в лесу, что не сильно сказывается на метрике.
Такой результат +- сходится с результатом, если судить по среднему результату на каждом из фолдов (средние score у победителей получились около 0.908).